In [1]:
%load_ext autoreload
%autoreload 2

%cd '..'

/dss/dsshome1/03/di93fup/polarization_reddit


In [2]:
import logging

# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s",
#     handlers=[
#         logging.FileHandler("data/logs/preprocess_comments_notebook.log"),
#         logging.StreamHandler()
#     ]
# )

In [12]:
import json

import numpy as np
import polars as pl
from tqdm import tqdm
tqdm.pandas()

from load.utils import (
    load_users,
    load_subreddits,
    load_user_party,
    load_comments,
    save_df_as_parquet,
    load_df_from_parquet,
)
from preprocessing.utils import (
    tokenize_comment,
    calculate_user_party,
    save_event_comments,
    build_vocab,
    save_event_vocab,
)

from preprocessing.constants import (
    METADATA_DIR,
    MASS_SHOOTINGS_EVENTS_INFO,
    MASS_SHOOTINGS_KEYWORDS,
    MIN_OCCURENCE_FOR_VOCAB,
    get_event_regex,
)

In [13]:
YEARS = [2015, 2016, 2017, 2018]
START_MONTH = 1
STOP_MONTH = 12

EVENT_NAMES = list(MASS_SHOOTINGS_EVENTS_INFO.keys())

MASS_SHOOTING_EVENTS_KEYWORDS = [
    keyword
    for event_name in EVENT_NAMES
    for keyword in MASS_SHOOTINGS_EVENTS_INFO[event_name]["keywords"]
]


In [16]:
MASS_SHOOTING_REGEX = get_event_regex(
    MASS_SHOOTINGS_KEYWORDS,
    MASS_SHOOTING_EVENTS_KEYWORDS,
    operator="and",
)

print(MASS_SHOOTING_REGEX)


(shoot|gun|kil|attack|massacr|victim)&(charleston|chattanoog|umpqu|roseburg|colorado springs|plan par|san bernardino|kalamazoo|orlando|puls nightclub|dalla|baton roug|burlington|cascad mal|fort lauderd|fresno|up|san francisco|vega|harvest fest|mandalay bay|rout 91|thornton|walmart|denv|sutherland springs|parkland|mars stonem|nashvil|waffl hous|sant fe|annapol|capit gazet|pittsburgh|tre of lif|thousand oak)


### User Affiliation

In [ ]:
subreddits = load_subreddits()

logging.info(subreddits.dtypes)

In [ ]:
# take into account network structure to find other partisan subreddits which are not labeled
dem_subreddits = set(
    json.load(
        open(
            f"{METADATA_DIR}/dem_subreddits_2016_unweighted.json",
        )
    )
)
rep_subreddits = set(
    json.load(
        open(
            f"{METADATA_DIR}/rep_subreddits_2016_unweighted.json",
        )
    )
)

partisan_subreddits = dem_subreddits | rep_subreddits


In [ ]:
# Filter partisan subreddits
# subreddits = subreddits[subreddits["party"].isin({"dem", "rep"})]
subreddits = subreddits.query("subreddit in @partisan_subreddits")

In [ ]:
logging.info(subreddits.groupby("party")["subreddit"].count())


In [ ]:
logging.info("Load comments...")

comments_pl = load_comments(
    years=YEARS,
    start_month=START_MONTH,
    stop_month=STOP_MONTH,
    engine="polars",
)

In [ ]:
partisan_comments_pl = comments_pl.filter(
    pl.col("subreddit").is_in(list(partisan_subreddits))
)


In [ ]:
partisan_comments = partisan_comments_pl.to_pandas(
).astype(
    {
        "author": "string[pyarrow]",
        "body_cleaned": "string[pyarrow]",
        "created_utc": "int64[pyarrow]",
        "subreddit": "string[pyarrow]",
    }
)


In [ ]:
partisan_comments.dtypes

In [ ]:
logging.info(partisan_comments.shape)


In [ ]:
logging.info("Add party information to comments...")

partisan_comments["party"] = np.where(
    partisan_comments["subreddit"].isin(dem_subreddits), "dem", "rep"
)
partisan_comments["party"] = partisan_comments["party"].astype("string[pyarrow]")


In [ ]:
partisan_comments.dtypes

In [ ]:
logging.info(partisan_comments.shape)

save_df_as_parquet(
    partisan_comments,
    target_file=f"partisan_comments_{YEARS}.parquet",
)


In [ ]:
# load_df_from_parquet(
#     file_name=f"partisan_comments_{YEARS}.parquet",
# )

In [ ]:
user_party = partisan_comments.groupby(by="author").progress_apply(
    calculate_user_party,
)
user_party = user_party[user_party["score"] != 0].reset_index().copy()
user_party["party"] = user_party["party"].astype("string[pyarrow]")


In [ ]:
user_party.dtypes

In [ ]:
logging.info(f"Nr of users: {len(user_party)}")

logging.info(user_party.groupby(by="party")["author"].count())

In [ ]:
save_df_as_parquet(
    data=user_party,
    target_file=f"user_party_{YEARS}.parquet",
)

logging.info(user_party.dtypes)

## Filter event comments

In [ ]:
# logging.info("Load user party")
# user_party = load_user_party(year=YEAR)

logging.info(user_party.dtypes)

In [ ]:
logging.info(partisan_comments.shape)

In [ ]:
users = load_users(engine="polars")

logging.info(users.dtypes)

In [ ]:
logging.info(users.shape)

In [ ]:
logging.info("Filter out bots & automoderators comments...")
partisan_comments = partisan_comments.merge(
    users,
    on="author",
    how="inner",
)

In [ ]:
logging.info(partisan_comments.dtypes)

In [ ]:
MASS_SHOOTING_EVENTS_KEYWORDS = [
    keyword
    for keyword in MASS_SHOOTINGS_EVENTS_INFO[event_name]["keywords"]
    for event_name in EVENT_NAMES
]


In [ ]:
logging.info("Filtering event data based on keywords...")
event_comments = partisan_comments[
    partisan_comments["body_cleaned"].str.contains(
        EVENT_INFO["regex"],
        regex=True,
    )
].copy()
logging.info("finished keyword filtering")

In [ ]:
logging.info(event_comments.shape)

In [ ]:
logging.info("save temp event comments")
save_event_comments(event_comments, "temp_mass_shootings")


In [ ]:
del user_party
del users
del subreddits
del comments_pl
del partisan_comments

In [ ]:
logging.info(event_comments.dtypes)

## Tokenize and stem comments

In [ ]:
logging.info("Tokenizing comments...")
event_comments["tokens"] = event_comments["body_cleaned"].progress_apply(
    tokenize_comment,
).astype("string[pyarrow]")
logging.info("Finish tokenizing comments")

In [ ]:
logging.info(event_comments.dtypes)

In [ ]:
logging.info(f"Nr of event comments: {len(event_comments)}")

In [ ]:
logging.info("saving event comments...")
save_event_comments(event_comments, "mass_shootings")

## Build event vocabulary

In [ ]:
# Read event data
# events_comments = load_event_comments(
#     event_comments,
#     EVENT_NAME,
#     file_type="parquet",
# )

event_vocab = build_vocab(
    event_comments["tokens"],
    min_comment_freq=MIN_OCCURENCE_FOR_VOCAB,
)


In [ ]:
logging.info("Vocabulary length")
logging.info(len(event_vocab))

In [ ]:
logging.info("Saving event vocab")
save_event_vocab(event_vocab, "mass_shootings")